In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.data import AUTOTUNE
from tensorflow.keras import layers, models

In [ ]:
#Masks.zip upload to Runtime

#!gdown 1lJq1OcLTpUPdOEwV7zMD83f2w40wLVBQ -O "/content/Masks.zip"

In [ ]:
#Images.zip new links
!gdown --id 1WQ6jcCh8ZTGE-got_w3CpgeYGUeYPZkc -O /content/images.zip


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1WQ6jcCh8ZTGE-got_w3CpgeYGUeYPZkc
From (redirected): https://drive.google.com/uc?id=1WQ6jcCh8ZTGE-got_w3CpgeYGUeYPZkc&confirm=t&uuid=3374a200-f61e-4ef6-b468-dd79e6e90599
To: /content/images.zip
100% 5.34G/5.34G [01:07<00:00, 79.3MB/s]


In [ ]:
!gdown --id 1ZbGWDf0X-ftjiXpSQ8Dgn0qvkkFj1gTS -O /content/mask.zip


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ZbGWDf0X-ftjiXpSQ8Dgn0qvkkFj1gTS
To: /content/mask.zip
100% 9.57M/9.57M [00:00<00:00, 30.1MB/s]


In [ ]:
#Click on the code cell output actions button and Hide the output after running this cell
!unzip mask.zip

Archive:  mask.zip
   creating: Masks/
  inflating: Masks/image_2.png       
  inflating: Masks/image_3.png       
  inflating: Masks/image_10.png      
  inflating: Masks/image_13.png      
  inflating: Masks/image_14.png      
  inflating: Masks/image_16.png      
  inflating: Masks/image_18.png      
  inflating: Masks/image_19.png      
  inflating: Masks/image_21.png      
  inflating: Masks/image_26.png      
  inflating: Masks/image_28.png      
  inflating: Masks/image_34.png      
  inflating: Masks/image_37.png      
  inflating: Masks/image_40.png      
  inflating: Masks/image_41.png      
  inflating: Masks/image_43.png      
  inflating: Masks/image_44.png      
  inflating: Masks/image_45.png      
  inflating: Masks/image_52.png      
  inflating: Masks/image_60.png      
  inflating: Masks/image_67.png      
  inflating: Masks/image_68.png      
  inflating: Masks/image_73.png      
  inflating: Masks/image_75.png      
  inflating: Masks/image_80.png      
  inflating

In [ ]:
#Click on the code cell output actions button and Hide the output after running this cell
!unzip images.zip

Archive:  images.zip
   creating: Images/
  inflating: Images/image_1.jpg      
  inflating: Images/image_3.jpg      
  inflating: Images/image_6.jpg      
  inflating: Images/image_7.jpg      
  inflating: Images/image_9.jpg      
  inflating: Images/image_11.jpg     
  inflating: Images/image_22.jpg     
  inflating: Images/image_29.jpg     
  inflating: Images/image_44.jpg     
  inflating: Images/image_46.jpg     
  inflating: Images/image_51.jpg     
  inflating: Images/image_68.jpg     
  inflating: Images/image_53.jpg     
  inflating: Images/image_57.jpg     
  inflating: Images/image_67.jpg     
  inflating: Images/image_89.jpg     
  inflating: Images/image_72.jpg     
  inflating: Images/image_90.jpg     
  inflating: Images/image_86.jpg     
  inflating: Images/image_94.jpg     
  inflating: Images/image_95.jpg     
  inflating: Images/image_97.jpg     
  inflating: Images/image_101.jpg    
  inflating: Images/image_106.jpg    
  inflating: Images/image_107.jpg    
  inflat

In [ ]:
# Define paths
IMAGE_DIR = "/content/Images"
MASK_DIR = "/content/Masks"
IMG_SIZE = (256, 256)         # Resize images to this size
BATCH_SIZE = 32

In [ ]:
# Function to load and preprocess an image and its corresponding mask
def load_image(image_path, mask_path):
    image = load_img(image_path, target_size=IMG_SIZE)
    mask = load_img(mask_path, target_size=IMG_SIZE, color_mode="grayscale")

    image = img_to_array(image) / 255.0  # Normalize to [0,1]
    mask = img_to_array(mask) #In our IEEE dataset, the masks are already normalized
#     mask = img_to_array(mask) / 255.0    # Normalize mask

    return image, mask


In [ ]:
# Get image and mask file paths
image_files = sorted([os.path.join(IMAGE_DIR, fname) for fname in os.listdir(IMAGE_DIR)])
mask_files = sorted([os.path.join(MASK_DIR, fname) for fname in os.listdir(MASK_DIR)])


In [ ]:
#!pip install transformers accelerate

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_classes = 2  # wildfire, background

feature_extractor = SegformerFeatureExtractor(reduce_labels=False)

model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/segformer-b0-finetuned-ade-512-512",
    num_labels=num_classes,
    ignore_mismatched_sizes=True
).to(device)

/usr/local/lib/python3.11/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/6.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/15.0M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([2]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([2, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
# --- Custom Dataset ---
class FireDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_transform = image_transform
        self.images = [file for file in os.listdir(image_dir) if file.endswith('.jpg') or file.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.jpg', '.png').replace('.jpeg', '.png'))

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.image_transform:
            image = self.image_transform(image)

        mask = transforms.Resize((256, 256))(mask)
        mask = torch.from_numpy(np.array(mask)).long()  # long for class indices
        mask = (mask > 0).long()  # Binary classification (background=0, fire=1)

        return image, mask

# --- Transforms ---
from transformers import SegformerFeatureExtractor
feature_extractor = SegformerFeatureExtractor(do_resize=True, size=256, do_normalize=True)

def segformer_transform(img):
    return feature_extractor(images=img, return_tensors="pt")["pixel_values"].squeeze()

# --- Paths ---
image_dir = './Images'
mask_dir = './Masks'

# --- Create Dataset ---
dataset = FireDataset(image_dir, mask_dir, image_transform=segformer_transform)

# --- Split ---
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# --- DataLoaders ---
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=8, shuffle=False)
test_loader = DataLoader(test_set, batch_size=8, shuffle=False)

In [ ]:
def weighted_bce_loss(output, target, weight=5):
    loss = F.binary_cross_entropy_with_logits(output, target, reduction='none')
    weights = 1 + weight * target
    return (loss * weights).mean()


In [ ]:
def train_segformer(model, dataloader, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for images, masks in tqdm(dataloader):
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(pixel_values=images, labels=masks)
        logits = outputs.logits  # shape: (B, 1, H, W)

        # Resize logits to match masks
        logits = F.interpolate(logits, size=masks.shape[-2:], mode='bilinear', align_corners=False)

        # Use only the foreground (fire) logit
        fire_logits = logits[:, 1, :, :].unsqueeze(1)  # (B, 1, H, W)

        # Apply sigmoid
        probs = torch.sigmoid(fire_logits)  # (B, 1, H, W)

        masks = masks.unsqueeze(1).float()  # (B, 1, H, W)

        loss = weighted_bce_loss(probs, masks, weight=5)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, jaccard_score

def evaluate_segformer(model, dataloader):
    model.eval()
    ious, f1s, precisions, recalls = [], [], [], []

    with torch.no_grad():
        for images, masks in tqdm(dataloader):
            images = images.to(device)
            masks = masks.to(device)  # shape: (B, H, W)

            outputs = model(pixel_values=images)
            logits = outputs.logits  # shape: (B, num_classes=2, h, w)

            # Resize logits to match mask size
            logits = F.interpolate(logits, size=masks.shape[-2:], mode='bilinear', align_corners=False)

            preds = torch.argmax(logits, dim=1)  # shape: (B, H, W)

            preds_np = preds.cpu().numpy().reshape(-1)
            masks_np = masks.cpu().numpy().reshape(-1)

            ious.append(jaccard_score(masks_np, preds_np, average='binary'))
            f1s.append(f1_score(masks_np, preds_np, average='binary'))
            precisions.append(precision_score(masks_np, preds_np, average='binary'))
            recalls.append(recall_score(masks_np, preds_np, average='binary'))

    return {
        "IoU": np.mean(ious),
        "F1": np.mean(f1s),
        "Precision": np.mean(precisions),
        "Recall": np.mean(recalls)
    }

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    train_loss = train_segformer(model, train_loader, optimizer, criterion)
    metrics = evaluate_segformer(model, val_loader)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val IoU: {metrics['IoU']:.4f}, F1: {metrics['F1']:.4f}, "
          f"Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}")


Epoch 1/10


100%|██████████| 38/38 [01:08<00:00,  1.80s/it]


Train Loss: 0.9005
Val IoU: 0.1300, F1: 0.2293, Precision: 0.1381, Recall: 0.7040

Epoch 2/10


100%|██████████| 38/38 [01:10<00:00,  1.85s/it]


Train Loss: 0.7998
Val IoU: 0.3017, F1: 0.4628, Precision: 0.3300, Recall: 0.7797

Epoch 3/10


...

Train Loss: 0.7130
Val IoU: 0.5601, F1: 0.7172, Precision: 0.6180, Recall: 0.8569

Epoch 9/10


100%|██████████| 38/38 [01:10<00:00,  1.84s/it]


Train Loss: 0.7109
Val IoU: 0.5731, F1: 0.7278, Precision: 0.6281, Recall: 0.8673

Epoch 10/10


100%|██████████| 38/38 [01:09<00:00,  1.83s/it]

Train Loss: 0.7098
Val IoU: 0.5970, F1: 0.7469, Precision: 0.6567, Recall: 0.8669


In [ ]:
torch.save(model.state_dict(), "segformer_finetuned.pth")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

def test_segformer(model, dataloader, threshold=0.5):
    model.load_state_dict(torch.load("segformer_finetuned.pth"))
    model.eval()
    all_preds = []
    all_targets = []
    total_loss = 0

    with torch.no_grad():
        for images, masks in tqdm(dataloader):
            images = images.to(device)
            masks = masks.to(device)  # (B, H, W)

            outputs = model(pixel_values=images)
            logits = outputs.logits  # (B, 2, H, W)

            # Resize logits to match target mask size
            logits = F.interpolate(logits, size=masks.shape[-2:], mode='bilinear', align_corners=False)
            fire_logits = logits[:, 1, :, :].unsqueeze(1)  # (B, 1, H, W)

            probs = torch.sigmoid(fire_logits)
            preds = (probs > threshold).float()  # binary mask

            masks = masks.unsqueeze(1).float()

            # Optional: compute loss
            loss = weighted_bce_loss(probs, masks, weight=5)
            total_loss += loss.item()

            all_preds.append(preds.cpu())
            all_targets.append(masks.cpu())

    # Stack all predictions and masks
    all_preds = torch.cat(all_preds).squeeze().numpy().flatten()
    all_targets = torch.cat(all_targets).squeeze().numpy().flatten()

    # Compute metrics
    iou = jaccard_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, zero_division=0)
    recall = recall_score(all_targets, all_preds, zero_division=0)
    f1 = f1_score(all_targets, all_preds, zero_division=0)
    acc = accuracy_score(all_targets, all_preds)

    tn, fp, fn, tp = confusion_matrix(all_targets, all_preds, labels=[0, 1]).ravel()
    fpr = fp / (fp + tn + 1e-8)
    fnr = fn / (fn + tp + 1e-8)

    print(f"\n--- Test Metrics ---")
    print(f"Loss: {total_loss / len(dataloader):.4f}")
    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"False Positive Rate: {fpr:.4f}")
    print(f"False Negative Rate: {fnr:.4f}")
    print(f"Pixel Accuracy: {acc:.4f}")

    return {
        "loss": total_loss / len(dataloader),
        "iou": iou,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "fpr": fpr,
        "fnr": fnr,
        "accuracy": acc
    }

In [ ]:
# Ensure model is on correct device (usually 'cuda' or 'mps')
model.to(device)

# Evaluate on test set
test_metrics = test_segformer(model, test_loader, threshold=0.5)

# Optional: print the returned metric dictionary
print(test_metrics)

100%|██████████| 38/38 [01:05<00:00,  1.72s/it]



--- Test Metrics ---
Loss: 0.7078
IoU: 0.6200
Precision: 0.6958
Recall: 0.8506
F1 Score: 0.7655
False Positive Rate: 0.0022
False Negative Rate: 0.1494
Pixel Accuracy: 0.9969
{'loss': 0.7078030595653936, 'iou': np.float64(0.6200421343898375), 'precision': 0.6957933141853031, 'recall': 0.8506401628878075, 'f1': 0.7654642076619399, 'fpr': np.float64(0.002198167220011719), 'fnr': np.float64(0.14935983711217954), 'accuracy': 0.9969371402778499}


In [ ]:
# reload and eval
model.load_state_dict(torch.load("segformer_finetuned.pth"))
model.eval()

SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  